## Zadanie 1

### Utwórz notebook Python i dołącz do niego niezbędne biblioteki.

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
print(spark.version)

23/05/29 12:01:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/29 12:01:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/29 12:01:18 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/05/29 12:01:18 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/05/29 12:01:18 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/05/29 12:01:18 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
23/05/29 12:01:18 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
23/05/29 12:01:18 WARN Utils: Service 'SparkUI' could

3.1.3


## Zadanie 2

### Obejrzyj zawartość pliku user_artist_data.txt w edytorze dla plików tekstowych Notatnik lub Notepad++ i porównaj z opisaną wyżej tabelką. Następnie napisz kod w języku Python, który odczyta zawartość pliku i zapamięta w kolekcji DataFrame systemu Spark.

In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

struct_uad = StructType([ \
StructField("UserId", IntegerType(), True),
StructField("ArtistId", IntegerType(), True),
StructField("Plays", IntegerType(), True)])

df_uad = spark.read.format("csv") \
.option("header", "false") \
.option("delimiter", " ") \
.schema(struct_uad) \
.load("user_artist_data.txt")

print(df_uad.count())
df_uad.show(10)

49481
+-------+--------+-----+
| UserId|ArtistId|Plays|
+-------+--------+-----+
|1059637| 1000010|  238|
|1059637| 1000049|    1|
|1059637| 1000056|    1|
|1059637| 1000062|   11|
|1059637| 1000094|    1|
|1059637| 1000112|  423|
|1059637| 1000113|    5|
|1059637| 1000114|    2|
|1059637| 1000123|    2|
|1059637| 1000130|19129|
+-------+--------+-----+
only showing top 10 rows



## Zadanie 3

### Analogicznie jak w poprzednim zadaniu odczytaj dane z pliku artist_data.txt. Uwaga, w tym pliku dane zostały rozdzielone znakiem tabulacji "\t".

In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

struct_ad = StructType([ \
StructField("ArtistId", IntegerType(), True),
StructField("ArtistName", StringType(), True)])

df_ad = spark.read.format("csv") \
.option("header", "false") \
.option("delimiter", "\t") \
.schema(struct_ad) \
.load("artist_data.txt")

print(df_ad.count())
df_ad.show(10)

30537
+--------+--------------------+
|ArtistId|          ArtistName|
+--------+--------------------+
| 1240105|        André Visior|
| 1240113|           riow arai|
| 1240132|Outkast & Rage Ag...|
| 6776115|            小松正夫|
| 1030848|      Raver's Nature|
| 6671601|      Erguner, Kudsi|
| 1106617|              Bloque|
| 1240185|      Lexy & K. Paul|
| 6671631|    Rev. W.M. Mosley|
| 6671632|      Labelle, Patti|
+--------+--------------------+
only showing top 10 rows



## Zadanie 4

### Wyświetl 10 pierwszych wierszy z kolekcji df_uad podstawiając nazwę artysty ArtistName zamiast jego identyfikatora ArtistId.

In [4]:
df_uad.join(df_ad, df_uad.ArtistId == df_ad.ArtistId, "inner"). \
 select("UserId", "ArtistName", "Plays").show(10)

+-------+--------------------+-----+
| UserId|          ArtistName|Plays|
+-------+--------------------+-----+
|1059637|           Aerosmith|  238|
|1059637|     Edna's Goldfish|    1|
|1059637|The Mighty Mighty...|    1|
|1059637|        Foo Fighters|   11|
|1059637|  The Bouncing Souls|    1|
|1059637|       Alkaline Trio|  423|
|1059637|         The Beatles|    5|
|1059637|           Pennywise|    2|
|1059637|             Incubus|    2|
|1059637|         Bright Eyes|19129|
+-------+--------------------+-----+
only showing top 10 rows



## Zadanie 5

### Sprawdź czy kolekcje df_uad i df_ad spełniają wymagania algorytmu ALS (Alternating Least Squares) z biblioteki MLlib pod względem zakresu danych. ALS wymaga, żeby identyfikatory zarówno użytkowników, jak i przedmiotów (tutaj artystów) mieściły się w zakresie liczby 32 bitowej.


In [7]:
print(pow(2,31))

df_uad.summary().show()

df_ad.summary().show()

2147483648
+-------+------------------+------------------+------------------+
|summary|            UserId|          ArtistId|             Plays|
+-------+------------------+------------------+------------------+
|  count|             49481|             49481|             49481|
|   mean|1328420.1949435137|2003155.0297285826|130.57579677047755|
| stddev| 452991.3131262286|2489609.3644763026|3034.3847545693047|
|    min|           1000647|                 1|                 1|
|    25%|           1024631|           1004015|                 2|
|    50%|           1059245|           1055553|                 7|
|    75%|           2010008|           1313001|                37|
|    max|           2288164|          10788218|            439771|
+-------+------------------+------------------+------------------+



+-------+------------------+--------------------+
|summary|          ArtistId|          ArtistName|
+-------+------------------+--------------------+
|  count|             30537|               30537|
|   mean|2723610.0883845827|            Infinity|
| stddev| 2875961.846523714|                 NaN|
|    min|                 1|      DAVID BLUMBERG|
|    25%|           1033156|                15.0|
|    50%|           1238776|               180.0|
|    75%|           2164466|               441.0|
|    max|          10788218|�����������������...|
+-------+------------------+--------------------+



## Zadanie 6

### Podziel dane z kolekcji df_uad na zbiór treningowy i testowy w proporcji 0.8 do 0.2. Wyświetl na ekranie liczbę elementów każdego ze zbiorów.

In [10]:
[df_train, df_test] = df_uad.randomSplit([0.8, 0.2])
print("Train:", df_train.count())
print("Test: ", df_test.count())

Train: 39503
Test:  9978


## Zadanie 7

### Zbuduj model dla sytemu rekomendacji korzystając z algorytmu ALS (Alternating Least Squares) z biblioteki MLlib oraz zbioru treningowego.

In [13]:
from pyspark.ml.recommendation import ALS

als = ALS( \
seed=327, \
implicitPrefs=True, \
rank = 10, \
maxIter = 5, \
userCol="UserId", \
itemCol="ArtistId", \
ratingCol="Plays", \
)
model = als.fit(df_train)

## Zadanie 8

### Uruchom model na zbiorze testowym. W zbiorze wynikowym usuń dane dla nieznanych id.

In [16]:
model.setColdStartStrategy("drop")
df_predictions= model.transform(df_test)

## Zadanie 9

### Wygeneruj 10 najlepszych znalezionych rekomendacji dla każdego użytkownika korzystając z metody recommendForAllUsers klasy modelu ALS. Wyświetl kilku pierwszych użytkowników i rekomendacje dla nich.

In [17]:
rec4user10 = model.recommendForAllUsers(10)
rec4user10.show(5,250)

+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| UserId|                                                                                                                                                                                                             recommendations|
+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|2007381|                           [{1006672, 2.3055282}, {1014421, 1.9727175}, {1428, 1.8304092}, {82, 1.8264846}, {831, 1.8027707}, {1, 1.7782328}, {1231740, 1.7767621}, {352, 1.7704989}, {53, 1.7531186}, {1000139, 1.7133775}]|
|1059637|                     [{1330, 2.3283498}, {1001027, 1.7579688}, {304

## Zadanie 10

### Wyświetl 5 najlepszych znalezionych rekomendacji dla dwóch użytkowników: 1059637 i 2007381. Skorzystaj z metody recommendForUserSubset.

In [19]:
usersId=[{'UserId':1059637},{'UserId':2007381}]
df_uid=spark.createDataFrame(usersId)
df_uid.show()

rec42users5 = model.recommendForUserSubset(df_uid, 5);
rec42users5.show(2,250)

+-------+
| UserId|
+-------+
|1059637|
|2007381|
+-------+



+-------+-------------------------------------------------------------------------------------------------------+
| UserId|                                                                                        recommendations|
+-------+-------------------------------------------------------------------------------------------------------+
|2007381|     [{1006672, 2.3055282}, {1014421, 1.9727175}, {1428, 1.8304092}, {82, 1.8264846}, {831, 1.8027707}]|
|1059637|[{1330, 2.3283498}, {1001027, 1.7579688}, {304, 1.7398098}, {1000323, 1.7247537}, {1004226, 1.7085555}]|
+-------+-------------------------------------------------------------------------------------------------------+



## Zadanie 11

### Wyświetl 6 rekomendacji dla jednego, wybranego użytkownika zamieniając identyfikatory artystów na ich nazwy.

In [26]:
from pyspark.sql.functions import explode

usersId=[{'UserId':1059637}]
df_uid=spark.createDataFrame(usersId)
df_uid.show()

df_uad.join(df_ad, df_uad.ArtistId == df_ad.ArtistId, "inner"). \
 select("UserId", "ArtistName", "Plays").show(10)

rec42users5 = model.recommendForUserSubset(df_uid, 6);
rec42users5.show(2,250)

+-------+
| UserId|
+-------+
|1059637|
+-------+

+-------+--------------------+-----+
| UserId|          ArtistName|Plays|
+-------+--------------------+-----+
|1059637|           Aerosmith|  238|
|1059637|     Edna's Goldfish|    1|
|1059637|The Mighty Mighty...|    1|
|1059637|        Foo Fighters|   11|
|1059637|  The Bouncing Souls|    1|
|1059637|       Alkaline Trio|  423|
|1059637|         The Beatles|    5|
|1059637|           Pennywise|    2|
|1059637|             Incubus|    2|
|1059637|         Bright Eyes|19129|
+-------+--------------------+-----+
only showing top 10 rows



+-------+----------------------------------------------------------------------------------------------------------------------------+
| UserId|                                                                                                             recommendations|
+-------+----------------------------------------------------------------------------------------------------------------------------+
|1059637|[{1330, 2.3283498}, {1001027, 1.7579688}, {304, 1.7398098}, {1000323, 1.7247537}, {1004226, 1.7085555}, {1001482, 1.690406}]|
+-------+----------------------------------------------------------------------------------------------------------------------------+



## Zadanie 12

### Sprawdź czy rekomendacje z zadania 9 są trafione wyświetlając nazwy artystów, których ten użytkownik najczęściej słuchał.

TypeError: condition should be string or Column

## Zadanie 13

### Wyświetl 8 najlepszych proponowanych rekomendacji użytkowników dla każdego artysty korzystając z metody recommendForAllItems. Ogranicz listę do 5 pierwszych artystów.

## Zadanie 15

### Wyświetl 4 najlepsze znalezione rekomendacje użytkowników dla trzech wybranych artystów: 1205, 1007027, 1007735. Skorzystaj z metody recommendForItemSubset.

## Zadanie 16

### Wyświetl 6 najlepszych rekomendacji użytkowników dla pięciu najczęściej słuchanych artystów.

## Zadanie 17

### Wyświetl 10 najlepszych rekomendacji dla 6 użytkowników którzy przesłuchali najwięcej utworów.


In [ ]:
Klasyfikacja, regresja, grupowania   (ALS)